<a href="https://colab.research.google.com/github/YasudaKaito/introduction_llm/blob/main/ch1/ch1_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[ja,sentencepiece,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-

In [ ]:
from transformers import pipeline

In [ ]:
# 文書分類
# 感情分析
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)
positive_text = "世界には言葉がわからなくても感動する音楽がある。"
print(text_classification_pipeline(positive_text)[0])

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

{'label': 'positive', 'score': 0.9993619322776794}


In [ ]:
negative_text = "世界には言葉がでないほどひどい音楽がある。"
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9636247754096985}


In [ ]:
# 自然言語推論NLI
# 含意／矛盾／中立を判定
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text = "二人の男性がジェット機を見ています"
entailment_text = "ジェット機を見ている人が二人います"
print(nli_pipeline({
    "text": text,
    "text_pair": entailment_text,
}))

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

{'label': 'entailment', 'score': 0.9964311122894287}


In [ ]:
contradiction_text = "二人の男性が飛んでいます"
print(nli_pipeline(
    {
        "text": text,
        "text_pair": contradiction_text,
    }
))

{'label': 'contradiction', 'score': 0.9990535378456116}


In [ ]:
neutral_text = "2人の男性が、白い飛行機を眺めています"
print(nli_pipeline(
    {
        "text": text,
        "text_pair": neutral_text,
    }
))

{'label': 'neutral', 'score': 0.9959145188331604}


In [ ]:
# 意味的類似度計算 STS
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply="none",
)
text = "川べりでサーフボードを持った人たちがいます"
sim_text = "サーファーたちが川べりに立っています"
result = text_sim_pipeline({
    "text": text,
    "text_pair": sim_text,
})
print(result)

config.json:   0%|          | 0.00/796 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

{'label': 'LABEL_0', 'score': 3.5703563690185547}


In [ ]:
dissim_text = "トイレの壁に黒いタオルがかけられています"
result = text_sim_pipeline({
    "text": text,
    "text_pair": dissim_text,
})
print(result)

{'label': 'LABEL_0', 'score': 0.041621580719947815}


In [ ]:
from torch.nn.functional import cosine_similarity

# 文埋め込み(sentenct embedding)
sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# text と sim_text のベクトルを取得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]

In [ ]:
text_emb.shape

torch.Size([768])

In [ ]:
# 類似度計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.8568588495254517


In [ ]:
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]

In [ ]:
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.45887044072151184


In [19]:
from pprint import pprint

# 固有表現認識(named entity recognition)
ner_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",
)
text = "大谷翔平は岩手県水沢市出身のプロ野球選手"
pprint(ner_pipeline(text))

config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'end': None,
  'entity_group': '人名',
  'score': 0.99823624,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9986874,
  'start': None,
  'word': '岩手 県 水沢 市'}]


In [20]:
# 要約
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [21]:
article = "ついに始まった３連休。テレビを見ながら過ごしている人も多いのではないだろうか？　今夜オススメなのは何と言っても、NHKスペシャル「世界を変えた男 スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。 今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界でのジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。 世界を変えた男　スティーブ・ジョブズ（NHKスペシャル）"
print(text2text_pipeline(article)[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'generated_text': '今夜はNHKスペシャル「世界を変えた男 スティーブ・ジョブズ」をチェック!'}


In [22]:
# トークナイゼーション
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")
tokenizer.tokenize("今日は天気が良いので")

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


['▁', '今日', 'は', '天気', 'が良い', 'の', 'で']

In [23]:
# テキスト生成
from transformers import AutoModelForCausalLM

In [24]:
model = AutoModelForCausalLM.from_pretrained(
    "abeja/gpt2-large-japanese"
)
inputs = tokenizer("今日は天気が良いので", return_tensors="pt")
inputs

config.json:   0%|          | 0.00/974 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

{'input_ids': tensor([[    7,  4639,    15, 17110, 16958,    10,    20,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [25]:
outputs = model.generate(
    **inputs,
    max_length=15,
    pad_token_id=tokenizer.pad_token_id,
)

In [26]:
outputs[0]

tensor([    7,  4639,    15, 17110, 16958,    10,    20,     2,   346,    20,
          196, 16002, 11671,  3426,     9])

In [27]:
generated_text = tokenizer.decode(
    outputs[0],
    skip_special_tokens=True,
)
print(generated_text)

今日は天気が良いので外でお弁当を食べました。
